In [2]:
import os

os.chdir('../')

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [4]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [ ]:
# This includes SLIM, RP3, EASER
old_similarity_matrix = sps.load_npz("SimilarityMatrixes/merging_models_final_similarity_TRAINVAL.npz")
# This includes graphConvolution Similarity Matrix
gC_similarity_matrix = sps.load_npz("SimilarityMatrixes/graphConvolution_W_sparse_TRAINVAL_num_factors:5645_alpha:_0.223329.npz")

In [ ]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

def objective_function(trial):

    alpha = trial.suggest_uniform('alpha', 0.0, 1.0)

    recommender = ItemKNNCustomSimilarityRecommender(URM_train_validation)

    new_similarity = old_similarity_matrix * alpha + gC_similarity_matrix * (1-alpha)

    assert type(new_similarity) is sps.csr_matrix

    recommender.fit(new_similarity)

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP

class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/Hybrid/hybrid_slim_EASER_GRAPHCONV.csv", index = False)
        
import optuna

optuna_study = optuna.create_study(study_name="hybrid_slim_rp3_easeR_graphConv", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100
                      )

# Let's just try linear combination

In [4]:
# This includes SLIM, RP3, EASER
old_similarity_matrix = sps.load_npz("SimilarityMatrixes/merging_models_final_similarity_TRAINVAL.npz")
# This includes graphConvolution Similarity Matrix
gC_similarity_matrix = sps.load_npz("SimilarityMatrixes/graphConvolution_W_sparse_TRAINVAL_num_factors:5645_alpha:_0.223329.npz")

In [5]:
print(type(old_similarity_matrix))
print(type(gC_similarity_matrix))

<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>


In [5]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

MM_recommend = ItemKNNCustomSimilarityRecommender(URM_train_validation)
MM_recommend.fit(old_similarity_matrix)

from Recommenders.Neural.Convolution.GraphConvolution import GraphConvolution

GC_recommend = GraphConvolution(URM_train_validation)
GC_recommend.W_sparse = gC_similarity_matrix

In [ ]:
from Recommenders.Hybrid.GeneralizedLinearCoupleHybridRecommender import GeneralizedLinearCoupleHybridRecommender
import optuna

recommenders = [MM_recommend, GC_recommend]

def objective_function(trial):
    recommender = GeneralizedLinearCoupleHybridRecommender(URM_train_validation, recommenders)
    alpha = trial.suggest_uniform('alpha', 0, 1)
    recommender.fit(alpha)
    result_dict, _ = evaluator_test.evaluateRecommender(recommender)
    MAP = result_dict.loc[10]["MAP"]
    return MAP

class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/Hybrid/hybrid_slim_EASER_GRAPHCONV_LINEARLY_COMBINED.csv", index = False)

optuna_study = optuna.create_study(study_name="HYBRID_IALS_INCLUDED", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100,
                      )

[I 2024-12-31 16:35:57,497] A new study created in memory with name: HYBRID_IALS_INCLUDED
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_24088/160973572.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0, 1)


EvaluatorHoldout: Processed 4000 (11.2%) in 5.81 min. Users per second: 11


In [6]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender
from Recommenders.Hybrid.GeneralizedLinearCoupleHybridRecommender import GeneralizedLinearCoupleHybridRecommender


slim_W = sps.load_npz("SimilarityMatrixes/slim_best_W.npz")
easeR_W = sps.load_npz("SimilarityMatrixes/easeR_best_W.npz")
alpha_slim_EASER = 0.622977719931239

similarity_SLIM_EASER = slim_W.multiply(alpha_slim_EASER) + easeR_W.multiply(1-alpha_slim_EASER)
del slim_W, easeR_W
# RP3
rp3_recommender = RP3betaRecommender(URM_all)
rp3_recommender.fit(topK=14, alpha=0.3683550822991944, beta=0.19877125816137325)

# Hybrid
final_recommender = ItemKNNCustomSimilarityRecommender(URM_all)
alpha = 0.8407690672301905
new_similarity = similarity_SLIM_EASER.multiply(alpha) + rp3_recommender.W_sparse.multiply(1-alpha)
del similarity_SLIM_EASER, rp3_recommender
final_recommender.fit(new_similarity)

sps.save_npz("SimilarityMatrixes/merging_models_final_similarity_SLIM_RP3_EASER.npz", final_recommender.W_sparse)

RP3betaRecommender: Similarity column 38121 (100.0%), 2946.02 column/sec. Elapsed time 12.94 sec
